In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.4.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.2
pandas 1.2.0
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(xtrain_all, ytrain_all), (xtest, ytest) = fashion_mnist.load_data()
xvalid, xtrain = xtrain_all[:5000], xtrain_all[5000:]
yvalid, ytrain = ytrain_all[:5000], ytrain_all[5000:]

print(xvalid.shape, yvalid.shape)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

In [ ]:
print(np.max(xtrain), np.min(xtrain))

In [ ]:
# Normalization
# x = (x - u) / std
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# xtrain [None, 28, 28] => [None, 784]
xtrain_scaled = scaler.fit_transform(
    xtrain.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xvalid_scaled = scaler.transform(
    xvalid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xtest_scaled = scaler.transform(
    xtest.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)

y_train = np.asarray(y_train, dtype = np.int64)
y_valid = np.asarray(y_valid, dtype = np.int64)
y_test = np.asarray(y_test, dtype = np.int64)

In [ ]:
print(np.max(xtrain_scaled), np.min(xtrain_scaled))

### 在归一化后构建 dataset

In [ ]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [ ]:
# 不能像 2.0 一样使用
batch_size = 20
epochs = 10
dataset = make_dataset(xtrain_scaled, ytrain, epochs = epochs, batch_size = batch_size)
for data, label in dataset.take(1):
    print(data)
    print(label)

In [ ]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train, epochs = epochs, batch_size = batch_size)

# 1. auto initialization
# 2. can't be re-initialized. make_initializable_iterator 可以
dataset_iter = dataset.make_one_shot_iterator()  # 创建一个遍历器
x, y = dataset_iter.get_next()  # 得到下一组数据
with tf.Session() as sess:
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)

In [ ]:
hidden_units = [100, 100]  # 2 layers of 100 hidden units each
class_num = 10

# x = tf.placeholder(tf.float32, [None, 28 * 28])  # None 代表 batch-size, 可以是任意值
# y = tf.placeholder(tf.inf64, [None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_nect_layer = tf.layers.dense(input_for_next_layer, hidden_unit, activation=tf.nn.relu)

# last hidden layer * W == (logits) -> softmax -> prob
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
logits = tf.layers.dense(input_for_next_layer, class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
print(x)
print(logits)

In [ ]:
# session

init = tf.global_variables_initializer()
train_steps_per_epoch = x_train.shape[0] // batch_size

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            loss_val, accuracy_val, _ = sess.run([loss, accuracy, train_op])
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f' % (
                epoch, step, loss_val, accuracy_val), end="")